In [12]:
import torch as th 
import numpy as np 
import random 
import cv2
import matplotlib.pyplot as plt 
random.seed(0)
np.random.seed(0)
th.manual_seed(0)
th.cuda.manual_seed(0)
th.backends.cudnn.deterministic = True

In [6]:
class StegNN(th.nn.Module):
    def __init__(self):
        super(StegNN, self).__init__()
        self.conv1 = th.nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, padding=2, stride=1) # 3x512x512 -> 8x512x512
        self.act_conv1 = th.nn.SiLU()
        self.pool1 = th.nn.MaxPool2d(kernel_size=5, padding=2, stride=4) # -> 8x128x128

        self.conv2 = th.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5, padding=2, stride=1) # -> 16x128x128
        self.act_conv2 = th.nn.SiLU()
        self.pool2 = th.nn.MaxPool2d(kernel_size=5, padding=2, stride=4) # -> 16x32x32
        
        self.conv3 = th.nn.Conv2d(in_channels=16, out_channels=64, kernel_size=5, padding=2, stride=1) # -> 64x32x32
        self.act_conv3 = th.nn.SiLU()
        self.pool3 = th.nn.MaxPool2d(kernel_size=5, padding=2, stride=4) # -> 64x8x8 

        self.fc1 = th.nn.Linear(8*8*64, 2048)
        self.act_lin1 = th.nn.Tanh()

        self.fc2 = th.nn.Linear(2048, 1024)
        self.act_lin2 = th.nn.Tanh()

        self.fc3 = th.nn.Linear(1024, 512)
        self.act_lin3 = th.nn.Tanh()

        self.fc4 = th.nn.Linear(512, 128)
        self.act_lin4 = th.nn.Tanh()

        self.fc5 = th.nn.Linear(128, 32)
        self.act_lin5 = th.nn.Tanh()

        self.fc6 = th.nn.Linear(32, 2) # 0 - ORIG 1 - ENCODE
        self.act_lin6 = th.nn.Tanh()
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.act_conv1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.act_conv2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.act_conv3(x)
        x = self.pool3(x)

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.fc1(x)
        x = self.act_lin1(x)

        x = self.fc2(x)
        x = self.act_lin2(x)

        x = self.fc3(x)
        x = self.act_lin3(x)

        x = self.fc4(x)
        x = self.act_lin4(x)

        x = self.fc5(x)
        x = self.act_lin5()

        x = self.fc6(x)

        return x 




In [23]:
stegNN = StegNN()


In [42]:
def train(net, X_train, y_train, X_test, y_test, batch_size=16, epoch_size=10, device='cpu'):
    net = net.to(device)
    loss = th.nn.CrossEntropyLoss()
    optimizer = th.optim.Adam(net.parameters(), lr=1.0e-3)

    test_accuracy_history = []
    test_loss_history = []
    train_loss_history = []

    
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    print(f'MODEL: {net.summary()} \n DEVICE: {device} \n BATCH_SIZE: {batch_size} \n EPOCH_SIZE: {epoch_size}') 

    for epoch in range(epoch_size):
        order = np.random.permutation(len(X_train))
        for start_index in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)  # forward pass 

            loss_value = loss(preds, y_batch)
            loss_value.backward()  # back propagation 
            train_loss_history.append(loss_value)
            optimizer.step() # step optimizer
            
            X_batch

        net.eval() # перевода модели в режим оценки

        test_preds = net.forward(X_test)
        test_loss_history.append(loss(test_preds, y_test).data) #.to(device))

        accuracy_test = (test_preds.argmax(dim=1) == y_test).float().mean().data #.to(device)
        test_accuracy_history.append(accuracy_test)

        

        print(f'Epoch: {epoch}. Accuracy: {round(accuracy_test, 4)}')


    return test_accuracy_history, test_loss_history, train_loss_history


In [25]:
def preprocess_image(img_path, target_size=(512, 512)):
         image = cv2.imread(img_path)
         h, w = image.shape[:2]
         if h > w:
             new_h = target_size[1]
             new_w = int(w * (new_h / h))
         else:
             new_w = target_size[0]
             new_h = int(h * (new_w / w))
         resized_image = cv2.resize(image, (new_w, new_h))
         result = np.full((target_size[1], target_size[0], 3), 255, dtype=np.uint8)  # создание белого изображение нужного размера
         result[(target_size[1]-new_h)//2:(target_size[1]-new_h)//2+new_h, (target_size[0]-new_w)//2:(target_size[0]-new_w)//2+new_w] = resized_image
         return result

In [39]:
image = cv2.imread('./0.png')
print(image.shape)
img = preprocess_image('./0.png')
img.shape
img = th.FloatTensor(img)
img /= 255.0
img

(669, 1000, 3)


tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])